In [33]:
import os

import numpy as np

# Computing neural network layers
import tensorflow as tf

# Parsing the .tsv/.csv files
import pandas as pd

#scikit learn
from sklearn.model_selection import train_test_split

#keras imports
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score

In [34]:
df = pd.read_csv('nbaDatabase2021-2022szn.csv')
df.head()

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_FG,homeTeam_3P,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,"October 19, 2021",Milwaukee Bucks,MIL,127,37.0,29.0,31.0,30.0,48-105,17-45,...,17-32,5,39,44,19,3,9,12,17,Milwaukee Bucks
1,"October 19, 2021",Los Angeles Lakers,LAL,114,34.0,25.0,26.0,29.0,45-95,15-42,...,14-39,9,41,50,30,9,2,17,18,Golden State Warriors
2,"October 20, 2021",Charlotte Hornets,CHA,123,27.0,32.0,33.0,31.0,46-107,13-31,...,17-47,8,43,51,29,2,10,16,24,Charlotte Hornets
3,"October 20, 2021",Detroit Pistons,DET,88,20.0,24.0,25.0,19.0,36-90,6-28,...,7-23,9,39,48,18,8,5,17,19,Chicago Bulls
4,"October 20, 2021",New York Knicks,NYK,138 (2OT),NaN,NaN,NaN,NaN,51-105,17-45,...,21-57,15,41,56,34,13,9,18,24,New York Knicks


In [35]:
print(len(df))
df = df.replace({np.nan: pd.NA, '': pd.NA, ' ': pd.NA}).applymap(lambda x: pd.NA if isinstance(x, float) and pd.isna(x) else x)
df = df.dropna(how='any')
print(len(df))

1323
1264


/var/folders/pw/f4_6q0l977bcqprlylly2d1w0000gq/T/ipykernel_1876/2452276954.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.replace({np.nan: pd.NA, '': pd.NA, ' ': pd.NA}).applymap(lambda x: pd.NA if isinstance(x, float) and pd.isna(x) else x)


In [36]:
df.head()

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_FG,homeTeam_3P,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,"October 19, 2021",Milwaukee Bucks,MIL,127,37.0,29.0,31.0,30.0,48-105,17-45,...,17-32,5,39,44,19,3,9,12,17,Milwaukee Bucks
1,"October 19, 2021",Los Angeles Lakers,LAL,114,34.0,25.0,26.0,29.0,45-95,15-42,...,14-39,9,41,50,30,9,2,17,18,Golden State Warriors
2,"October 20, 2021",Charlotte Hornets,CHA,123,27.0,32.0,33.0,31.0,46-107,13-31,...,17-47,8,43,51,29,2,10,16,24,Charlotte Hornets
3,"October 20, 2021",Detroit Pistons,DET,88,20.0,24.0,25.0,19.0,36-90,6-28,...,7-23,9,39,48,18,8,5,17,19,Chicago Bulls
5,"October 20, 2021",Toronto Raptors,TOR,83,18.0,19.0,22.0,24.0,30-97,7-34,...,5-27,7,44,51,20,14,8,21,17,Washington Wizards


In [37]:
df['homeTeam_points_total'] = df['homeTeam_points_total'].astype(int)
df['awayTeam_points_total'] = df['awayTeam_points_total'].astype(int)

In [38]:
df['winner'] = np.where(
    df['homeTeam_points_total'] > df['awayTeam_points_total'], 0,
    np.where(df['homeTeam_points_total'] < df['awayTeam_points_total'], 1, 2) 
)
df.head()

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_FG,homeTeam_3P,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,"October 19, 2021",Milwaukee Bucks,MIL,127,37.0,29.0,31.0,30.0,48-105,17-45,...,17-32,5,39,44,19,3,9,12,17,0
1,"October 19, 2021",Los Angeles Lakers,LAL,114,34.0,25.0,26.0,29.0,45-95,15-42,...,14-39,9,41,50,30,9,2,17,18,1
2,"October 20, 2021",Charlotte Hornets,CHA,123,27.0,32.0,33.0,31.0,46-107,13-31,...,17-47,8,43,51,29,2,10,16,24,0
3,"October 20, 2021",Detroit Pistons,DET,88,20.0,24.0,25.0,19.0,36-90,6-28,...,7-23,9,39,48,18,8,5,17,19,1
5,"October 20, 2021",Toronto Raptors,TOR,83,18.0,19.0,22.0,24.0,30-97,7-34,...,5-27,7,44,51,20,14,8,21,17,1


In [39]:
df['homeTeam_FG'] = df['homeTeam_FG'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list) and len(x) == 2 else np.nan)

df['awayTeam_FG'] = df['awayTeam_FG'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list) and len(x) == 2 else np.nan)

In [40]:
df.head()

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_FG,homeTeam_3P,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,"October 19, 2021",Milwaukee Bucks,MIL,127,37.0,29.0,31.0,30.0,0.457143,17-45,...,17-32,5,39,44,19,3,9,12,17,0
1,"October 19, 2021",Los Angeles Lakers,LAL,114,34.0,25.0,26.0,29.0,0.473684,15-42,...,14-39,9,41,50,30,9,2,17,18,1
2,"October 20, 2021",Charlotte Hornets,CHA,123,27.0,32.0,33.0,31.0,0.429907,13-31,...,17-47,8,43,51,29,2,10,16,24,0
3,"October 20, 2021",Detroit Pistons,DET,88,20.0,24.0,25.0,19.0,0.400000,6-28,...,7-23,9,39,48,18,8,5,17,19,1
5,"October 20, 2021",Toronto Raptors,TOR,83,18.0,19.0,22.0,24.0,0.309278,7-34,...,5-27,7,44,51,20,14,8,21,17,1


In [41]:
df['homeTeam_3P'] = df['homeTeam_3P'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list) and len(x) == 2 else np.nan)

df['awayTeam_3P'] = df['awayTeam_3P'].str.split('-').apply(lambda x: int(x[0]) / int(x[1]) if isinstance(x, list) and len(x) == 2 else np.nan)
df.head()

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_FG,homeTeam_3P,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,"October 19, 2021",Milwaukee Bucks,MIL,127,37.0,29.0,31.0,30.0,0.457143,0.377778,...,0.531250,5,39,44,19,3,9,12,17,0
1,"October 19, 2021",Los Angeles Lakers,LAL,114,34.0,25.0,26.0,29.0,0.473684,0.357143,...,0.358974,9,41,50,30,9,2,17,18,1
2,"October 20, 2021",Charlotte Hornets,CHA,123,27.0,32.0,33.0,31.0,0.429907,0.419355,...,0.361702,8,43,51,29,2,10,16,24,0
3,"October 20, 2021",Detroit Pistons,DET,88,20.0,24.0,25.0,19.0,0.400000,0.214286,...,0.304348,9,39,48,18,8,5,17,19,1
5,"October 20, 2021",Toronto Raptors,TOR,83,18.0,19.0,22.0,24.0,0.309278,0.205882,...,0.185185,7,44,51,20,14,8,21,17,1


In [42]:
df['homeTeam'] = 0
df['awayTeam'] = 1
df.head()

,date,homeTeam,homeTeam_id,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_FG,homeTeam_3P,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,"October 19, 2021",0,MIL,127,37.0,29.0,31.0,30.0,0.457143,0.377778,...,0.531250,5,39,44,19,3,9,12,17,0
1,"October 19, 2021",0,LAL,114,34.0,25.0,26.0,29.0,0.473684,0.357143,...,0.358974,9,41,50,30,9,2,17,18,1
2,"October 20, 2021",0,CHA,123,27.0,32.0,33.0,31.0,0.429907,0.419355,...,0.361702,8,43,51,29,2,10,16,24,0
3,"October 20, 2021",0,DET,88,20.0,24.0,25.0,19.0,0.400000,0.214286,...,0.304348,9,39,48,18,8,5,17,19,1
5,"October 20, 2021",0,TOR,83,18.0,19.0,22.0,24.0,0.309278,0.205882,...,0.185185,7,44,51,20,14,8,21,17,1


In [43]:
df = df.drop(columns=['date', 'homeTeam_id', 'awayTeam_id'])
df.head()

,homeTeam,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,homeTeam_FG,homeTeam_3P,homeTeam_O_Reb,homeTeam_D_Reb,...,awayTeam_3P,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner
0,0,127,37.0,29.0,31.0,30.0,0.457143,0.377778,13,41,...,0.531250,5,39,44,19,3,9,12,17,0
1,0,114,34.0,25.0,26.0,29.0,0.473684,0.357143,5,40,...,0.358974,9,41,50,30,9,2,17,18,1
2,0,123,27.0,32.0,33.0,31.0,0.429907,0.419355,12,34,...,0.361702,8,43,51,29,2,10,16,24,0
3,0,88,20.0,24.0,25.0,19.0,0.400000,0.214286,11,36,...,0.304348,9,39,48,18,8,5,17,19,1
5,0,83,18.0,19.0,22.0,24.0,0.309278,0.205882,19,35,...,0.185185,7,44,51,20,14,8,21,17,1


In [52]:
df = df[df['winner'] != 2]

In [53]:
X = df.drop(columns=['winner']) 
y = df['winner']
y.head()


0    0
1    1
2    0
3    1
5    1
Name: winner, dtype: int64

In [56]:
print(y.values)

[0 1 0 ... 1 0 1]


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [58]:
print("Unique values in y_train:", y_train.unique())
print("Unique values in y_test:", y_test.unique())

Unique values in y_train: [1 0]
Unique values in y_test: [1 0]


In [59]:
#Define the deep learning model
model = Sequential()
model.add(Dense(128, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# EarlyStopping callback
earlystopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

# ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath='best_model_weights2.weights.h5', save_weights_only=True, monitor='val_accuracy', save_best_only=True, verbose=1)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=1000, batch_size=64, validation_data=(X_test, y_test), callbacks=[checkpoint, earlystopping])

Epoch 1/1000
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4939 - loss: 0.9248 
Epoch 1: val_accuracy improved from -inf to 0.38340, saving model to best_model_weights2.weights.h5
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.5043 - loss: 0.9035 - val_accuracy: 0.3834 - val_loss: 1.3212
Epoch 2/1000
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8084 - loss: 0.4276 
Epoch 2: val_accuracy improved from 0.38340 to 0.85771, saving model to best_model_weights2.weights.h5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8105 - loss: 0.4243 - val_accuracy: 0.8577 - val_loss: 0.3829
Epoch 3/1000
15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9245 - loss: 0.2672 
Epoch 3: val_accuracy improved from 0.85771 to 0.86561, saving model to best_model_weights2.weights.h5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9257 - loss: 0.2642 - val_accuracy: 0.8656 - val_loss: 0.3226
Epoch 4/1000
14/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9729 - loss: 0.1661